# Model comparison


In [ ]:
%load_ext dotenv
%dotenv
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from typing import Dict
from functools import partial

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.offsetbox as moffsetbox
import numpy as np
import pandas as pd
import xarray as xr

from src.analysis import (
    calc_shf_slurb,
    calc_shf_usm,
    calc_qsws_slurb,
    calc_qsws_usm,
    calc_us_slurb,
    calc_us_usm,
    calc_t_can_slurb,
    calc_t_can_usm,
    calc_ta_2m_slurb,
    calc_ta_2m_usm,
    calc_rh_can_slurb,
    calc_rh_can_usm,
    time_to_datetimeindex,
    bbox_filter,
    time_filter,
    lcz_filter,
    map_func_to_resolutions,
    map_func_to_lcz_and_time_periods,
)

from src.plotting import FIGURE_WIDTH_L

from src.config import get_config, get_dask_cluster
from src.job_generation import read_namelist

config = get_config()
cluster, client = get_dask_cluster(config)

In [ ]:
plt.rcParams["font.family"] = config.plotting.font_family
plt.rcParams["font.serif"] = config.plotting.font_serif
plt.rcParams["font.size"] = config.plotting.font_size

## Analysis preparation


Define target areas for spatial averaging. For definitions, see 04-model_setups.ipynb.


In [ ]:
urban_offset = (2110 + 2 * 896 - 1536, 1280)

# bbox for spatial averaging area used for xy-averaged analysis
# corresponds to inner 16 LCZ tiles
total_target_area_bbox = (
    urban_offset[0] + 256,
    urban_offset[1] + 256,
    urban_offset[0] + 1536 - 256,
    urban_offset[1] + 1536 - 256,
)

total_urban_area_bbox = (
    urban_offset[0],
    urban_offset[1],
    urban_offset[0] + 1536,
    urban_offset[1] + 1536,
)

Define the area as a filter.


In [ ]:
total_target_area_filter = partial(bbox_filter, bbox=total_target_area_bbox)
total_urban_area_filter = partial(bbox_filter, bbox=total_urban_area_bbox)

In [ ]:
daytime_filter = partial(
    time_filter,
    start_time=np.datetime64("2018-03-31T12:00"),
    end_time=np.datetime64("2018-03-31T16:00"),
)
nighttime_filter = partial(
    time_filter,
    start_time=np.datetime64("2018-04-01T00:00"),
    end_time=np.datetime64("2018-04-01T04:00"),
)

Load LCZ mask and offset to root origin.


In [ ]:
lcz_map = xr.open_dataset(Path(config.path.data.raw) / "lcz" / "lcz_map.nc")["lcz"]
lcz_map["x"] = lcz_map["x"] + urban_offset[0]
lcz_map["y"] = lcz_map["y"] + urban_offset[1]

Set origin datetime.


In [ ]:
shared_p3d = read_namelist(
    Path(config.path.experiments.comparison) / "shared_coarse_p3d.yml"
)
origin_date_time = pd.Timestamp(
    shared_p3d["initialization_parameters"]["origin_date_time"]
)

Load data output files.


In [ ]:
domain_identifiers = {
    "coarse": "",
    "medium-coarse": "N02",
    "medium-fine": "N03",
    "fine": "N04",
}
domain_offsets = {
    "coarse": (0.0, 0.0),
    "medium-coarse": (448.0, 256.0),
    "medium-fine": (896.0, 512.0),
    "fine": (2112.0, 1280.0),
}
dataset_types = (
    "av_3d",
    "av_xy",
    "av_xz",
    "av_yz",
    "xy",
    "surf",
    "av_surf",
    "pr",
    "ts",
)


def drop_allnan_time(ds):
    # A function to drop invalid times from output, PALM doesn't write a fill value for some reason.
    return ds.where(ds["time"] < 1e36, drop=True)

In [ ]:
slurb_run = "slurb"
usm_run = "usm"

In [ ]:
outputs = {
    slurb_run: {"coarse": {}, "medium-coarse": {}, "medium-fine": {}},
    usm_run: {"coarse": {}, "medium-coarse": {}, "medium-fine": {}, "fine": {}},
}
static_input = {
    slurb_run: {"coarse": None, "medium-coarse": None, "medium-fine": None},
    usm_run: {"coarse": None, "medium-coarse": None, "medium-fine": None, "fine": None},
}
for model in (slurb_run, usm_run):
    path = Path(config.path.data.jobs) / ("slurb_c_" + model) / "OUTPUT"
    for domain, ident in domain_identifiers.items():
        for dataset in dataset_types:
            ident_str = f"_{ident}" if ident != "" else ""
            try:
                ds = xr.open_mfdataset(
                    str(path) + f"/slurb_c_{model}_{dataset}{ident_str}.*.nc",
                    chunks={"time": "auto"},
                    decode_times=False,
                    preprocess=drop_allnan_time,
                    parallel=True,
                    combine="nested",
                    concat_dim="time",
                )
            except OSError:
                continue
            ds = time_to_datetimeindex(ds, origin_date_time)
            # Offset aggregation period labels to period center
            ds["time"] = ds["time"] - pd.Timedelta(15, "m")
            ds["second_of_day"] = (
                ds.time.dt.hour * 3600 + ds.time.dt.minute * 60 + ds.time.dt.second
            )
            # ds = ds.where(ds["second_of_day"] < 41760.0, drop=True)
            for coord in (
                "x",
                "xu",
                "x_yz",
                "xu_yz",
            ):
                if coord in ds.coords.keys():
                    ds[coord] = ds[coord] + domain_offsets[domain][0]
            for coord in ("y", "yv", "y_xz", "yv_xz"):
                if coord in ds.coords.keys():
                    ds[coord] = ds[coord] + domain_offsets[domain][1]
            if "xs" in ds.data_vars.keys():
                ds["xs"] = ds["xs"] + domain_offsets[domain][0]
            if "xs" in ds.data_vars.keys():
                ds["ys"] = ds["ys"] + domain_offsets[domain][1]
            ds = ds.drop_duplicates(..., keep="first")
            outputs[model][domain][dataset] = ds

    # Load static input
    path = Path(config.path.data.jobs) / ("slurb_c_" + model) / "INPUT"
    for domain, ident in domain_identifiers.items():
        ident_str = f"_{ident}" if ident != "" else ""
        try:
            ds = xr.open_dataset(
                str(path) + f"/slurb_c_{model}_static{ident_str}",
                decode_times=False,
                engine="netcdf4",
            )
        except OSError:
            continue
        ds["x"] = ds["x"] + domain_offsets[domain][0]
        ds["y"] = ds["y"] + domain_offsets[domain][1]
        static_input[model][domain] = ds

## Diurnal cycle analysis


In [ ]:
# Some helpers for plotting and computation
resolutions_slurb = {"coarse": 16, "medium-coarse": 8, "medium-fine": 4}
resolutions_usm = {"coarse": 16, "medium-coarse": 8, "medium-fine": 4, "fine": 2}
plot_style = {
    "coarse": {
        "label": r"SLUrb, $\Delta_i=16~\mathrm{m}$",
        "linestyle": "--",
        "color": "tab:blue",
    },
    "medium-coarse": {
        "label": r"SLUrb, $\Delta_i=8~\mathrm{m}$",
        "linestyle": "--",
        "color": "tab:blue",
    },
}

### Heat fluxes


In [ ]:
func = partial(calc_shf_slurb, filters=[total_target_area_filter])
shf_ts_slurb = map_func_to_resolutions(func, outputs[slurb_run], resolutions_slurb)

func = partial(calc_qsws_slurb, filters=[total_target_area_filter])
qsws_ts_slurb = map_func_to_resolutions(func, outputs[slurb_run], resolutions_slurb)

In [ ]:
func = partial(calc_shf_usm, filters=[total_target_area_filter], h_canopy=24.0)
shf_ts_usm = map_func_to_resolutions(func, outputs[usm_run], resolutions_usm)

func = partial(calc_qsws_usm, filters=[total_target_area_filter], h_canopy=24.0)
qsws_ts_usm = map_func_to_resolutions(func, outputs[usm_run], resolutions_usm)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(17.5 / 2.56, 4), constrained_layout=True)
(shf_ts_slurb[0] + qsws_ts_slurb[0]).plot.line(
    ax=ax, color="tab:blue", label=r"SLUrb, $\Delta_i=16~\mathrm{m}$", linestyle="--"
)
(shf_ts_slurb[1] + qsws_ts_slurb[1]).plot.line(
    ax=ax, color="tab:orange", label=r"SLUrb, $\Delta_i=8~\mathrm{m}$", linestyle="--"
)
(shf_ts_slurb[2] + qsws_ts_slurb[2]).plot.line(
    ax=ax, color="tab:green", label=r"SLUrb, $\Delta_i=4~\mathrm{m}$", linestyle="--"
)

(shf_ts_usm[0] + qsws_ts_usm[0]).plot.line(
    ax=ax, color="tab:blue", label=r"Resolved, $\Delta_i=16~\mathrm{m}$", linestyle=":"
)
(shf_ts_usm[1] + qsws_ts_usm[1]).plot.line(
    ax=ax, color="tab:orange", label=r"Resolved, $\Delta_i=8~\mathrm{m}$", linestyle=":"
)
(shf_ts_usm[2] + qsws_ts_usm[2]).plot.line(
    ax=ax, color="tab:green", label=r"Resolved, $\Delta_i=4~\mathrm{m}$", linestyle=":"
)
(shf_ts_usm[3] + qsws_ts_usm[3]).plot.line(
    ax=ax, color="k", label=r"Resolved, $\Delta_i=2~\mathrm{m}$"
)
ax.set_title("")
ax.set_ylabel("Total heat flux (W m$^-2$)")
ax.xaxis.set_major_locator(mdates.HourLocator(interval=3))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
ax.grid()
ax.legend()
out_path = Path(config.path.results)
out_path.mkdir(parents=True, exist_ok=True)
ax.set_xlim((pd.Timestamp("2018-03-31 04:00:00"), pd.Timestamp("2018-04-01 04:00:00")))
heatflux_ylims = ax.get_ylim()
fig.savefig(out_path / "model_comparison_total_heat.pdf", dpi=300)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(17.5 / 2.56, 4), constrained_layout=True)
shf_ts_slurb[0].plot.line(
    ax=ax, color="tab:blue", label=r"SLUrb, $\Delta_i=16~\mathrm{m}$", linestyle="--"
)
shf_ts_slurb[1].plot.line(
    ax=ax, color="tab:orange", label=r"SLUrb, $\Delta_i=8~\mathrm{m}$", linestyle="--"
)
shf_ts_slurb[2].plot.line(
    ax=ax, color="tab:green", label=r"SLUrb, $\Delta_i=4~\mathrm{m}$", linestyle="--"
)

shf_ts_usm[0].plot.line(
    ax=ax, color="tab:blue", label=r"Resolved, $\Delta_i=16~\mathrm{m}$", linestyle=":"
)
shf_ts_usm[1].plot.line(
    ax=ax, color="tab:orange", label=r"Resolved, $\Delta_i=8~\mathrm{m}$", linestyle=":"
)
shf_ts_usm[2].plot.line(
    ax=ax, color="tab:green", label=r"Resolved, $\Delta_i=4~\mathrm{m}$", linestyle=":"
)
shf_ts_usm[3].plot.line(ax=ax, color="k", label=r"Resolved, $\Delta_i=2~\mathrm{m}$")
ax.set_title("")
ax.set_ylabel("Sensible heat flux (W m$^-2$)")
ax.xaxis.set_major_locator(mdates.HourLocator(interval=3))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
ax.grid()
ax.legend()
ax.set_ylim(heatflux_ylims)
out_path = Path(config.path.results)
out_path.mkdir(parents=True, exist_ok=True)
ax.set_xlim((pd.Timestamp("2018-03-31 04:00:00"), pd.Timestamp("2018-04-01 04:00:00")))
fig.savefig(out_path / "model_comparison_sensible_heat.pdf", dpi=300)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(17.5 / 2.56, 4), constrained_layout=True)
qsws_ts_slurb[0].plot.line(
    ax=ax, color="tab:blue", label=r"SLUrb, $\Delta_i=16~\mathrm{m}$", linestyle="--"
)
qsws_ts_slurb[1].plot.line(
    ax=ax, color="tab:orange", label=r"SLUrb, $\Delta_i=8~\mathrm{m}$", linestyle="--"
)
qsws_ts_slurb[2].plot.line(
    ax=ax, color="tab:green", label=r"SLUrb, $\Delta_i=4~\mathrm{m}$", linestyle="--"
)

qsws_ts_usm[0].plot.line(
    ax=ax, color="tab:blue", label=r"Resolved, $\Delta_i=16~\mathrm{m}$", linestyle=":"
)
qsws_ts_usm[1].plot.line(
    ax=ax, color="tab:orange", label=r"Resolved, $\Delta_i=8~\mathrm{m}$", linestyle=":"
)
qsws_ts_usm[2].plot.line(
    ax=ax, color="tab:green", label=r"Resolved, $\Delta_i=4~\mathrm{m}$", linestyle=":"
)
qsws_ts_usm[3].plot.line(ax=ax, color="k", label=r"Resolved, $\Delta_i=2~\mathrm{m}$")
ax.set_title("")
ax.set_ylabel("Latent heat flux (W m$^-2$)")
ax.xaxis.set_major_locator(mdates.HourLocator(interval=3))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
ax.grid()
ax.legend()
ax.set_ylim(heatflux_ylims)
out_path = Path(config.path.results)
out_path.mkdir(parents=True, exist_ok=True)
ax.set_xlim((pd.Timestamp("2018-03-31 04:00:00"), pd.Timestamp("2018-04-01 00:00:00")))
fig.savefig(out_path / "model_comparison_latent_heat.pdf", dpi=300)

### Friction velocity


In [ ]:
func = partial(calc_us_slurb, filters=[total_target_area_filter])
us_ts_slurb = map_func_to_resolutions(func, outputs[slurb_run], resolutions_slurb)

In [ ]:
func = partial(calc_us_usm, filters=[total_target_area_filter])
us_ts_usm = map_func_to_resolutions(func, outputs[usm_run], resolutions_usm)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(17.5 / 2.56, 4), constrained_layout=True)
us_ts_slurb[0].plot.line(
    ax=ax, color="tab:blue", label=r"SLUrb, $\Delta_i=16~\mathrm{m}$", linestyle="--"
)
us_ts_slurb[1].plot.line(
    ax=ax, color="tab:orange", label=r"SLUrb, $\Delta_i=8~\mathrm{m}$", linestyle="--"
)
us_ts_slurb[2].plot.line(
    ax=ax, color="tab:green", label=r"SLUrb, $\Delta_i=4~\mathrm{m}$", linestyle="--"
)

us_ts_usm[0].plot.line(
    ax=ax, color="tab:blue", label=r"Resolved, $\Delta_i=16~\mathrm{m}$", linestyle=":"
)
us_ts_usm[1].plot.line(
    ax=ax, color="tab:orange", label=r"Resolved, $\Delta_i=8~\mathrm{m}$", linestyle=":"
)
us_ts_usm[2].plot.line(
    ax=ax, color="tab:green", label=r"Resolved, $\Delta_i=4~\mathrm{m}$", linestyle=":"
)
us_ts_usm[3].plot.line(ax=ax, color="k", label=r"Resolved, $\Delta_i=2~\mathrm{m}$")
ax.set_ylim((0.0, None))
ax.grid()
ax.set_ylabel("Friction velocity (m/s)")
ax.set_title("")
ax.xaxis.set_major_locator(mdates.HourLocator(interval=3))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
ax.legend()
out_path = Path(config.path.results)
out_path.mkdir(parents=True, exist_ok=True)
ax.set_xlim((pd.Timestamp("2018-03-31 04:00:00"), pd.Timestamp("2018-04-01 04:00:00")))
fig.savefig(out_path / "model_comparison_friction_velocity.pdf", dpi=300)

### Canyon air tempertature


In [ ]:
func = partial(calc_t_can_slurb, filters=[total_target_area_filter])
t_can_ts_slurb = map_func_to_resolutions(func, outputs[slurb_run], resolutions_slurb)

In [ ]:
func = partial(calc_t_can_usm, filters=[total_target_area_filter], h_bld=14.6)
t_can_ts_usm = map_func_to_resolutions(func, outputs[usm_run], resolutions_usm)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(17.5 / 2.56, 4), constrained_layout=True)
t_can_ts_slurb[0].plot.line(
    ax=ax, color="tab:blue", label=r"SLUrb, $\Delta_i=16~\mathrm{m}$", linestyle="--"
)
t_can_ts_slurb[1].plot.line(
    ax=ax, color="tab:orange", label=r"SLUrb, $\Delta_i=8~\mathrm{m}$", linestyle="--"
)
t_can_ts_slurb[2].plot.line(
    ax=ax, color="tab:green", label=r"SLUrb, $\Delta_i=4~\mathrm{m}$", linestyle="--"
)

t_can_ts_usm[0].plot.line(
    ax=ax, color="tab:blue", label=r"Resolved, $\Delta_i=16~\mathrm{m}$", linestyle=":"
)
t_can_ts_usm[1].plot.line(
    ax=ax, color="tab:orange", label=r"Resolved, $\Delta_i=8~\mathrm{m}$", linestyle=":"
)
t_can_ts_usm[2].plot.line(
    ax=ax, color="tab:green", label=r"Resolved, $\Delta_i=4~\mathrm{m}$", linestyle=":"
)
t_can_ts_usm[3].plot.line(ax=ax, color="k", label=r"Resolved, $\Delta_i=2~\mathrm{m}$")

ax.grid()
ax.set_ylabel("Canyon air temperature (K)")
ax.set_title("")
ax.xaxis.set_major_locator(mdates.HourLocator(interval=3))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
ax.legend()
out_path = Path(config.path.results)
out_path.mkdir(parents=True, exist_ok=True)
ax.set_xlim((pd.Timestamp("2018-03-31 04:00:00"), pd.Timestamp("2018-04-01 04:00:00")))

### Canyon relative humidity


In [ ]:
func = partial(calc_rh_can_slurb, filters=[total_target_area_filter])
rh_can_ts_slurb = map_func_to_resolutions(func, outputs[slurb_run], resolutions_slurb)

In [ ]:
func = partial(calc_rh_can_usm, filters=[total_target_area_filter], h_bld=14.6)
rh_can_ts_usm = map_func_to_resolutions(func, outputs[usm_run], resolutions_usm)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(17.5 / 2.56, 4), constrained_layout=True)
rh_can_ts_slurb[0].plot.line(
    ax=ax, color="tab:blue", label=r"SLUrb, $\Delta_i=16~\mathrm{m}$", linestyle="--"
)
rh_can_ts_slurb[1].plot.line(
    ax=ax, color="tab:orange", label=r"SLUrb, $\Delta_i=8~\mathrm{m}$", linestyle="--"
)
rh_can_ts_slurb[2].plot.line(
    ax=ax, color="tab:green", label=r"SLUrb, $\Delta_i=4~\mathrm{m}$", linestyle="--"
)

rh_can_ts_usm[0].plot.line(
    ax=ax, color="tab:blue", label=r"Resolved, $\Delta_i=16~\mathrm{m}$", linestyle=":"
)
rh_can_ts_usm[1].plot.line(
    ax=ax, color="tab:orange", label=r"Resolved, $\Delta_i=8~\mathrm{m}$", linestyle=":"
)
rh_can_ts_usm[2].plot.line(
    ax=ax, color="tab:green", label=r"Resolved, $\Delta_i=4~\mathrm{m}$", linestyle=":"
)
rh_can_ts_usm[3].plot.line(ax=ax, color="k", label=r"Resolved, $\Delta_i=2~\mathrm{m}$")

ax.grid()
ax.set_ylabel(r"$RH_\mathrm{canyon}$ (%)")
ax.set_title("")
ax.xaxis.set_major_locator(mdates.HourLocator(interval=3))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
ax.legend()
out_path = Path(config.path.results)
out_path.mkdir(parents=True, exist_ok=True)
ax.set_xlim((pd.Timestamp("2018-03-31 04:00:00"), pd.Timestamp("2018-04-01 04:00:00")))

### Multi-panel diurnal comparison plot

A multi-panel plot of diurnals for the manuscript as calculated before.


In [ ]:
fig = plt.figure(figsize=(FIGURE_WIDTH_L, 16.5 / 2.56), layout="constrained")
gs = fig.add_gridspec(
    4, 2, height_ratios=[0.3, 1, 1, 1], width_ratios=[1, 1], wspace=0.0, hspace=0.0
)

legend_ax = fig.add_subplot(gs[0, 0:])
legend_ax.set_axis_off()

ax1 = fig.add_subplot(gs[1, 0])
ax2 = fig.add_subplot(gs[1, 1])
ax3 = fig.add_subplot(gs[2, 0])
ax4 = fig.add_subplot(gs[2, 1])
ax5 = fig.add_subplot(gs[3, 0])
ax6 = fig.add_subplot(gs[3, 1])


slurb_coarse_line = mlines.Line2D(
    (0,),
    (0,),
    linewidth=1.5,
    color="tab:blue",
    linestyle="--",
    label=r"SLUrb, $\Delta_i=16~\mathrm{m}$",
)
slurb_medium_coarse_line = mlines.Line2D(
    (0,),
    (0,),
    linewidth=1.5,
    color="tab:orange",
    linestyle="--",
    label=r"SLUrb, $\Delta_i=8~\mathrm{m}$",
)
slurb_medium_fine_line = mlines.Line2D(
    (0,),
    (0,),
    linewidth=1.5,
    color="tab:green",
    linestyle="--",
    label=r"SLUrb, $\Delta_i=4~\mathrm{m}$",
)

usm_coarse_line = mlines.Line2D(
    (0,),
    (0,),
    linewidth=1.5,
    color="tab:blue",
    linestyle=":",
    label=r"Resolved, $\Delta_i=16~\mathrm{m}$",
)
usm_medium_coarse_line = mlines.Line2D(
    (0,),
    (0,),
    linewidth=1.5,
    color="tab:orange",
    linestyle=":",
    label=r"Resolved, $\Delta_i=8~\mathrm{m}$",
)
usm_medium_fine_line = mlines.Line2D(
    (0,),
    (0,),
    linewidth=1.5,
    color="tab:green",
    linestyle=":",
    label=r"Resolved, $\Delta_i=4~\mathrm{m}$",
)
usm_fine_line = mlines.Line2D(
    (0,),
    (0,),
    linewidth=1.5,
    color="k",
    linestyle="-",
    label=r"Resolved, $\Delta_i=2~\mathrm{m}$",
)

legend = legend_ax.legend(
    handles=[
        usm_coarse_line,
        slurb_coarse_line,
        usm_medium_coarse_line,
        slurb_medium_coarse_line,
        usm_medium_fine_line,
        slurb_medium_fine_line,
        usm_fine_line,
    ],
    loc="center",
    bbox_to_anchor=(0.5, 0.5),
    borderaxespad=0,
    ncol=4,
)


(shf_ts_slurb[0] + qsws_ts_slurb[0]).plot.line(
    ax=ax1, color="tab:blue", linestyle="--", _labels=False
)
(shf_ts_slurb[1] + qsws_ts_slurb[1]).plot.line(
    ax=ax1, color="tab:orange", linestyle="--", _labels=False
)
(shf_ts_slurb[2] + qsws_ts_slurb[2]).plot.line(
    ax=ax1, color="tab:green", linestyle="--", _labels=False
)
(shf_ts_usm[0] + qsws_ts_usm[0]).plot.line(
    ax=ax1, color="tab:blue", linestyle=":", _labels=False
)
(shf_ts_usm[1] + qsws_ts_usm[1]).plot.line(
    ax=ax1, color="tab:orange", linestyle=":", _labels=False
)
(shf_ts_usm[2] + qsws_ts_usm[2]).plot.line(
    ax=ax1, color="tab:green", linestyle=":", _labels=False
)
(shf_ts_usm[3] + qsws_ts_usm[3]).plot.line(ax=ax1, color="k", _labels=False)


us_ts_slurb[0].plot.line(
    ax=ax2,
    color="tab:blue",
    linestyle="--",
    _labels=False,
)
us_ts_slurb[1].plot.line(
    ax=ax2,
    color="tab:orange",
    linestyle="--",
    _labels=False,
)
us_ts_slurb[2].plot.line(
    ax=ax2,
    color="tab:green",
    linestyle="--",
    _labels=False,
)
us_ts_usm[0].plot.line(
    ax=ax2,
    color="tab:blue",
    linestyle=":",
    _labels=False,
)
us_ts_usm[1].plot.line(
    ax=ax2,
    color="tab:orange",
    linestyle=":",
    _labels=False,
)
us_ts_usm[2].plot.line(
    ax=ax2,
    color="tab:green",
    linestyle=":",
    _labels=False,
)
us_ts_usm[3].plot.line(
    ax=ax2,
    color="k",
    _labels=False,
)


shf_ts_slurb[0].plot.line(
    ax=ax3,
    color="tab:blue",
    linestyle="--",
    _labels=False,
)
shf_ts_slurb[1].plot.line(
    ax=ax3,
    color="tab:orange",
    linestyle="--",
    _labels=False,
)
shf_ts_slurb[2].plot.line(
    ax=ax3,
    color="tab:green",
    linestyle="--",
    _labels=False,
)
shf_ts_usm[0].plot.line(
    ax=ax3,
    color="tab:blue",
    linestyle=":",
    _labels=False,
)
shf_ts_usm[1].plot.line(
    ax=ax3,
    color="tab:orange",
    linestyle=":",
    _labels=False,
)
shf_ts_usm[2].plot.line(
    ax=ax3,
    color="tab:green",
    linestyle=":",
    _labels=False,
)
shf_ts_usm[3].plot.line(
    ax=ax3,
    color="k",
    _labels=False,
)

qsws_ts_slurb[0].plot.line(
    ax=ax4,
    color="tab:blue",
    linestyle="--",
    _labels=False,
)
qsws_ts_slurb[1].plot.line(
    ax=ax4,
    color="tab:orange",
    linestyle="--",
    _labels=False,
)
qsws_ts_slurb[2].plot.line(
    ax=ax4,
    color="tab:green",
    linestyle="--",
    _labels=False,
)
qsws_ts_usm[0].plot.line(
    ax=ax4,
    color="tab:blue",
    linestyle=":",
    _labels=False,
)
qsws_ts_usm[1].plot.line(
    ax=ax4,
    color="tab:orange",
    linestyle=":",
    _labels=False,
)
qsws_ts_usm[2].plot.line(
    ax=ax4,
    color="tab:green",
    linestyle=":",
    _labels=False,
)
qsws_ts_usm[3].plot.line(ax=ax4, color="k", _labels=False)


(t_can_ts_slurb[0] - 273.15).plot.line(
    ax=ax5,
    color="tab:blue",
    linestyle="--",
    _labels=False,
)
(t_can_ts_slurb[1] - 273.15).plot.line(
    ax=ax5,
    color="tab:orange",
    linestyle="--",
    _labels=False,
)
(t_can_ts_slurb[2] - 273.15).plot.line(
    ax=ax5,
    color="tab:green",
    linestyle="--",
    _labels=False,
)
(t_can_ts_usm[0] - 273.15).plot.line(
    ax=ax5,
    color="tab:blue",
    linestyle=":",
    _labels=False,
)
(t_can_ts_usm[1] - 273.15).plot.line(
    ax=ax5,
    color="tab:orange",
    linestyle=":",
    _labels=False,
)
(t_can_ts_usm[2] - 273.15).plot.line(
    ax=ax5,
    color="tab:green",
    linestyle=":",
    _labels=False,
)
(t_can_ts_usm[3] - 273.15).plot.line(ax=ax5, color="k", _labels=False)

rh_can_ts_slurb[0].plot.line(
    ax=ax6,
    color="tab:blue",
    linestyle="--",
    _labels=False,
)
rh_can_ts_slurb[1].plot.line(
    ax=ax6,
    color="tab:orange",
    linestyle="--",
    _labels=False,
)
rh_can_ts_slurb[2].plot.line(
    ax=ax6,
    color="tab:green",
    linestyle="--",
    _labels=False,
)
rh_can_ts_usm[0].plot.line(
    ax=ax6,
    color="tab:blue",
    linestyle=":",
    _labels=False,
)
rh_can_ts_usm[1].plot.line(
    ax=ax6,
    color="tab:orange",
    linestyle=":",
    _labels=False,
)
rh_can_ts_usm[2].plot.line(
    ax=ax6,
    color="tab:green",
    linestyle=":",
    _labels=False,
)
rh_can_ts_usm[3].plot.line(ax=ax6, color="k", _labels=False)


ax1.set_ylabel(r"$H+LE$ (W m$^{-2}$)")
ax2.set_ylabel(r"$u_*$ (m s$^{-1}$)")
ax3.set_ylabel(r"$H$ (W m$^{-2}$)")
ax4.set_ylabel(r"$LE$ (W m$^{-2}$)")
ax5.set_ylabel(r"$T_{\mathrm{can}}$ ($^\circ\mathrm{C}$)")
ax6.set_ylabel(r"$\mathrm{RH}_{\mathrm{can}}$ (%)")

ax2.set_ylim(0, None)
hflux_lims = ax1.get_ylim()
ax3.set_ylim(hflux_lims)
ax4.set_ylim(hflux_lims)

ax1.add_artist(moffsetbox.AnchoredText("(a)", loc="upper left", pad=0.2, borderpad=0))
ax2.add_artist(moffsetbox.AnchoredText("(b)", loc="upper left", pad=0.2, borderpad=0))
ax3.add_artist(moffsetbox.AnchoredText("(c)", loc="upper left", pad=0.2, borderpad=0))
ax4.add_artist(moffsetbox.AnchoredText("(d)", loc="upper left", pad=0.2, borderpad=0))
ax5.add_artist(moffsetbox.AnchoredText("(e)", loc="upper left", pad=0.2, borderpad=0))
ax6.add_artist(moffsetbox.AnchoredText("(f)", loc="upper left", pad=0.2, borderpad=0))


for ax in (ax1, ax2, ax3, ax4, ax5, ax6):
    ax.xaxis.set_major_locator(mdates.HourLocator(interval=4))
    ax.xaxis.set_major_formatter(mdates.DateFormatter(""))
    ax.xaxis.set_label("")
    ax.set_xlim(
        (pd.Timestamp("2018-03-31 04:00:00"), pd.Timestamp("2018-04-01 04:00:00"))
    )
    ax.grid()

for ax in (ax5, ax6):
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%H"))
    ax.set_xlabel("Time (local/solar)")

out_path = Path(config.path.results)
out_path.mkdir(parents=True, exist_ok=True)
fig.savefig(out_path / "model_comparison_dirunals.pdf", dpi=300)

Compute some additional values/results for the manuscript.


In [ ]:
# Ratio of 16 m and 2 m mean diurnal ustar.
print(f"Resolved u_* 16 m: {us_ts_usm[0].mean().values:.2f}")
print(f"Resolved u_* 2 m: {us_ts_usm[3].mean().values:.2f}")

print(
    f"SLUrb daytime u_* 16 m: {us_ts_slurb[0].where(daytime_filter).mean().values:.2f}"
)
print(
    f"Resolved daytime u_* 2 m: {us_ts_usm[3].where(daytime_filter).mean().values:.2f}"
)

print(
    f"SLUrb nighttime u_* 16 m: {us_ts_slurb[0].where(nighttime_filter).mean().values:.2f}"
)
print(
    f"Resolved nighttime u_* 2 m: {us_ts_usm[3].where(nighttime_filter).mean().values:.2f}"
)

## Day/night resolution sensitivity analysis based on LCZ


Sample individual LCZ tiles.


In [ ]:
lcz_2_filter = partial(
    lcz_filter,
    target_area=total_target_area_bbox,
    n_tiles=4,
    tile_size=(256.0, 256.0),
    lcz=2,
)
lcz_5_filter = partial(
    lcz_filter,
    target_area=total_target_area_bbox,
    n_tiles=4,
    tile_size=(256.0, 256.0),
    lcz=5,
)

In [ ]:
lcz_filters = [lcz_2_filter, lcz_5_filter, total_target_area_filter]
time_filters = [daytime_filter, nighttime_filter]

In [ ]:
shf_ts_slurb = map_func_to_lcz_and_time_periods(
    calc_shf_slurb, lcz_filters, time_filters, outputs[slurb_run], resolutions_slurb
)
shf_ts_usm = map_func_to_lcz_and_time_periods(
    calc_shf_usm,
    lcz_filters,
    time_filters,
    outputs[usm_run],
    resolutions_usm,
    h_canopy=24.0,
)

In [ ]:
qsws_ts_slurb = map_func_to_lcz_and_time_periods(
    calc_qsws_slurb, lcz_filters, time_filters, outputs[slurb_run], resolutions_slurb
)
qsws_ts_usm = map_func_to_lcz_and_time_periods(
    calc_qsws_usm,
    lcz_filters,
    time_filters,
    outputs[usm_run],
    resolutions_usm,
    h_canopy=24.0,
)

In [ ]:
us_ts_slurb = map_func_to_lcz_and_time_periods(
    calc_us_slurb, lcz_filters, time_filters, outputs[slurb_run], resolutions_slurb
)
us_ts_usm = map_func_to_lcz_and_time_periods(
    calc_us_usm, lcz_filters, time_filters, outputs[usm_run], resolutions_usm
)

In [ ]:
t_can_ts_slurb = map_func_to_lcz_and_time_periods(
    calc_t_can_slurb, lcz_filters, time_filters, outputs[slurb_run], resolutions_slurb
)
t_can_ts_usm = map_func_to_lcz_and_time_periods(
    partial(calc_t_can_usm, h_bld=14.6),
    lcz_filters,
    time_filters,
    outputs[usm_run],
    resolutions_usm,
)

In [ ]:
fig = plt.figure(figsize=(FIGURE_WIDTH_L, 16.5 / 2.56), layout="constrained")
gs = fig.add_gridspec(
    4, 2, height_ratios=[0.2, 1, 1, 1], width_ratios=[1, 1], wspace=0.0, hspace=0.0
)

legend_ax = fig.add_subplot(gs[0, 0:])
legend_ax.set_axis_off()

ax1 = fig.add_subplot(gs[1, 0])
ax2 = fig.add_subplot(gs[1, 1])
ax3 = fig.add_subplot(gs[2, 0])
ax4 = fig.add_subplot(gs[2, 1])
ax5 = fig.add_subplot(gs[3, 0])
ax6 = fig.add_subplot(gs[3, 1])

ax1.set_title("Daytime")
ax2.set_title("Nighttime")

color1 = "#a7226e"
color2 = "#2f9599"
color3 = "k"

slurb_line = mlines.Line2D(
    (0,), (0,), linewidth=1.5, color="k", linestyle="--", marker="x", label=r"SLUrb"
)
usm_line = mlines.Line2D(
    (0,),
    (0,),
    linewidth=1.5,
    color="k",
    linestyle=":",
    marker="o",
    fillstyle="none",
    label=r"Resolved",
)

total_urban_line = mlines.Line2D(
    (0,), (0,), linewidth=1.5, color=color3, linestyle="-", label=r"Total urban"
)

lcz_2_line = mlines.Line2D(
    (0,), (0,), linewidth=1.5, color=color1, linestyle="-", label=r"LCZ 2"
)
lcz_5_line = mlines.Line2D(
    (0,), (0,), linewidth=1.5, color=color2, linestyle="-", label=r"LCZ 5"
)
legend = legend_ax.legend(
    handles=[slurb_line, usm_line, total_urban_line, lcz_2_line, lcz_5_line],
    loc="center",
    bbox_to_anchor=(0.5, 0.5),
    borderaxespad=0,
    ncol=5,
)

ax1.plot(
    resolutions_slurb.values(),
    [
        arr1.mean() + arr2.mean()
        for arr1, arr2 in zip(shf_ts_slurb[0][0], qsws_ts_slurb[0][0])
    ],
    color=color1,
    linestyle="--",
    marker="x",
    lw=1.5,
)
ax1.plot(
    resolutions_usm.values(),
    [
        arr1.mean() + arr2.mean()
        for arr1, arr2 in zip(shf_ts_usm[0][0], qsws_ts_usm[0][0])
    ],
    color=color1,
    linestyle=":",
    marker="o",
    fillstyle="none",
    lw=1.5,
)
ax1.plot(
    resolutions_slurb.values(),
    [
        arr1.mean() + arr2.mean()
        for arr1, arr2 in zip(shf_ts_slurb[0][1], qsws_ts_slurb[0][1])
    ],
    color=color2,
    linestyle="--",
    marker="x",
    lw=1.5,
)
ax1.plot(
    resolutions_usm.values(),
    [
        arr1.mean() + arr2.mean()
        for arr1, arr2 in zip(shf_ts_usm[0][1], qsws_ts_usm[0][1])
    ],
    color=color2,
    linestyle=":",
    marker="o",
    fillstyle="none",
    lw=1.5,
)
ax1.plot(
    resolutions_slurb.values(),
    [
        arr1.mean() + arr2.mean()
        for arr1, arr2 in zip(shf_ts_slurb[0][2], qsws_ts_slurb[0][2])
    ],
    color=color3,
    linestyle="--",
    marker="x",
    lw=1.5,
)
ax1.plot(
    resolutions_usm.values(),
    [
        arr1.mean() + arr2.mean()
        for arr1, arr2 in zip(shf_ts_usm[0][2], qsws_ts_usm[0][2])
    ],
    color=color3,
    linestyle=":",
    marker="o",
    fillstyle="none",
    lw=1.5,
)


ax2.plot(
    resolutions_slurb.values(),
    [
        arr1.mean() + arr2.mean()
        for arr1, arr2 in zip(shf_ts_slurb[1][0], qsws_ts_slurb[1][0])
    ],
    color=color1,
    linestyle="--",
    marker="x",
    lw=1.5,
)
ax2.plot(
    resolutions_usm.values(),
    [
        arr1.mean() + arr2.mean()
        for arr1, arr2 in zip(shf_ts_usm[1][0], qsws_ts_usm[1][0])
    ],
    color=color1,
    linestyle=":",
    marker="o",
    fillstyle="none",
    lw=1.5,
)
ax2.plot(
    resolutions_slurb.values(),
    [
        arr1.mean() + arr2.mean()
        for arr1, arr2 in zip(shf_ts_slurb[1][1], qsws_ts_slurb[1][1])
    ],
    color=color2,
    linestyle="--",
    marker="x",
    lw=1.5,
)
ax2.plot(
    resolutions_usm.values(),
    [
        arr1.mean() + arr2.mean()
        for arr1, arr2 in zip(shf_ts_usm[1][1], qsws_ts_usm[1][1])
    ],
    color=color2,
    linestyle=":",
    marker="o",
    fillstyle="none",
    lw=1.5,
)
ax2.plot(
    resolutions_slurb.values(),
    [
        arr1.mean() + arr2.mean()
        for arr1, arr2 in zip(shf_ts_slurb[1][2], qsws_ts_slurb[1][2])
    ],
    color=color3,
    linestyle="--",
    marker="x",
    lw=1.5,
)
ax2.plot(
    resolutions_usm.values(),
    [
        arr1.mean() + arr2.mean()
        for arr1, arr2 in zip(shf_ts_usm[1][2], qsws_ts_usm[1][2])
    ],
    color=color3,
    linestyle=":",
    marker="o",
    fillstyle="none",
    lw=1.5,
)


ax3.plot(
    resolutions_slurb.values(),
    [arr.mean() for arr in us_ts_slurb[0][0]],
    color=color1,
    linestyle="--",
    marker="x",
    lw=1.5,
)
ax3.plot(
    resolutions_usm.values(),
    [arr.mean() for arr in us_ts_usm[0][0]],
    color=color1,
    linestyle=":",
    marker="o",
    fillstyle="none",
    lw=1.5,
)
ax3.plot(
    resolutions_slurb.values(),
    [arr.mean() for arr in us_ts_slurb[0][1]],
    color=color2,
    linestyle="--",
    marker="x",
    lw=1.5,
)
ax3.plot(
    resolutions_usm.values(),
    [arr.mean() for arr in us_ts_usm[0][1]],
    color=color2,
    linestyle=":",
    marker="o",
    fillstyle="none",
    lw=1.5,
)
ax3.plot(
    resolutions_slurb.values(),
    [arr.mean() for arr in us_ts_slurb[0][2]],
    color=color3,
    linestyle="--",
    marker="x",
    lw=1.5,
)
ax3.plot(
    resolutions_usm.values(),
    [arr.mean() for arr in us_ts_usm[0][2]],
    color=color3,
    linestyle=":",
    marker="o",
    fillstyle="none",
    lw=1.5,
)

ax4.plot(
    resolutions_slurb.values(),
    [arr.mean() for arr in us_ts_slurb[1][0]],
    color=color1,
    linestyle="--",
    marker="x",
    lw=1.5,
)
ax4.plot(
    resolutions_usm.values(),
    [arr.mean() for arr in us_ts_usm[1][0]],
    color=color1,
    linestyle=":",
    marker="o",
    fillstyle="none",
    lw=1.5,
)
ax4.plot(
    resolutions_slurb.values(),
    [arr.mean() for arr in us_ts_slurb[1][1]],
    color=color2,
    linestyle="--",
    marker="x",
    lw=1.5,
)
ax4.plot(
    resolutions_usm.values(),
    [arr.mean() for arr in us_ts_usm[1][1]],
    color=color2,
    linestyle=":",
    marker="o",
    fillstyle="none",
    lw=1.5,
)
ax4.plot(
    resolutions_slurb.values(),
    [arr.mean() for arr in us_ts_slurb[1][2]],
    color=color3,
    linestyle="--",
    marker="x",
    lw=1.5,
)
ax4.plot(
    resolutions_usm.values(),
    [arr.mean() for arr in us_ts_usm[1][2]],
    color=color3,
    linestyle=":",
    marker="o",
    fillstyle="none",
    lw=1.5,
)


(
    ax5.plot(
        resolutions_slurb.values(),
        [(arr.mean() - 273.15) for arr in t_can_ts_slurb[0][0]],
        color=color1,
        linestyle="--",
        marker="x",
        lw=1.5,
    ),
)
ax5.plot(
    resolutions_usm.values(),
    [(arr.mean() - 273.15) for arr in t_can_ts_usm[0][0]],
    color=color1,
    linestyle=":",
    marker="o",
    fillstyle="none",
    lw=1.5,
)
ax5.plot(
    resolutions_slurb.values(),
    [(arr.mean() - 273.15) for arr in t_can_ts_slurb[0][1]],
    color=color2,
    linestyle="--",
    marker="x",
    lw=1.5,
)
ax5.plot(
    resolutions_usm.values(),
    [(arr.mean() - 273.15) for arr in t_can_ts_usm[0][1]],
    color=color2,
    linestyle=":",
    marker="o",
    fillstyle="none",
    lw=1.5,
)
ax5.plot(
    resolutions_slurb.values(),
    [(arr.mean() - 273.15) for arr in t_can_ts_slurb[0][2]],
    color=color3,
    linestyle="--",
    marker="x",
    lw=1.5,
)
ax5.plot(
    resolutions_usm.values(),
    [(arr.mean() - 273.15) for arr in t_can_ts_usm[0][2]],
    color=color3,
    linestyle=":",
    marker="o",
    fillstyle="none",
    lw=1.5,
)


(
    ax6.plot(
        resolutions_slurb.values(),
        [(arr.mean() - 273.15) for arr in t_can_ts_slurb[1][0]],
        color=color1,
        linestyle="--",
        marker="x",
        lw=1.5,
    ),
)
ax6.plot(
    resolutions_usm.values(),
    [(arr.mean() - 273.15) for arr in t_can_ts_usm[1][0]],
    color=color1,
    linestyle=":",
    marker="o",
    fillstyle="none",
    lw=1.5,
)
ax6.plot(
    resolutions_slurb.values(),
    [(arr.mean() - 273.15) for arr in t_can_ts_slurb[1][1]],
    color=color2,
    linestyle="--",
    marker="x",
    lw=1.5,
)
ax6.plot(
    resolutions_usm.values(),
    [(arr.mean() - 273.15) for arr in t_can_ts_usm[1][1]],
    color=color2,
    linestyle=":",
    marker="o",
    fillstyle="none",
    lw=1.5,
)
ax6.plot(
    resolutions_slurb.values(),
    [(arr.mean() - 273.15) for arr in t_can_ts_slurb[1][2]],
    color=color3,
    linestyle="--",
    marker="x",
    lw=1.5,
)
ax6.plot(
    resolutions_usm.values(),
    [(arr.mean() - 273.15) for arr in t_can_ts_usm[1][2]],
    color=color3,
    linestyle=":",
    marker="o",
    fillstyle="none",
    lw=1.5,
)


# Scale the axes to same relative range
for ax_left, ax_right in ((ax1, ax2), (ax5, ax6)):
    ymin1, ymax1 = ax_left.get_ylim()
    ymin2, ymax2 = ax_right.get_ylim()

    center1, range1 = (ymin1 + ymax1) / 2, (ymax1 - ymin1) / 2
    center2, range2 = (ymin2 + ymax2) / 2, (ymax2 - ymin2) / 2

    max_range = max(range1, range2)

    new_ylim1 = (center1 - max_range, center1 + max_range)
    new_ylim2 = (center2 - max_range, center2 + max_range)

    ax_left.set_ylim(new_ylim1)
    ax_right.set_ylim(new_ylim2)


ax3.set_ylim((0, ax3.get_ylim()[1]))
ax4.set_ylim((0, ax3.get_ylim()[1]))

ax1.set_ylabel(r"$H+LE$ (W m$^{-2}$)")
ax3.set_ylabel(r"$u_*$ (m s$^{-1}$)")
ax5.set_ylabel(r"$T_{\mathrm{can}}$ ($^\circ\mathrm{C}$)")


ax1.add_artist(moffsetbox.AnchoredText("(a)", loc="upper left", pad=0.2, borderpad=0))
ax2.add_artist(moffsetbox.AnchoredText("(b)", loc="upper left", pad=0.2, borderpad=0))
ax3.add_artist(moffsetbox.AnchoredText("(c)", loc="upper left", pad=0.2, borderpad=0))
ax4.add_artist(moffsetbox.AnchoredText("(d)", loc="upper left", pad=0.2, borderpad=0))
ax5.add_artist(moffsetbox.AnchoredText("(e)", loc="upper left", pad=0.2, borderpad=0))
ax6.add_artist(moffsetbox.AnchoredText("(f)", loc="upper left", pad=0.2, borderpad=0))


for ax in (ax1, ax2, ax3, ax4, ax5, ax6):
    ax.set_xticks([16, 8, 4, 2])
    ax.set_xlim((17, 1))
    ax.grid()

for ax in (ax1, ax2, ax3, ax4):
    plt.setp(ax.get_xticklabels(), visible=False)
    ax.xaxis.set_label("")

for ax in (ax2, ax4, ax6):
    # plt.setp(ax.get_yticklabels(), visible=False)
    ax.yaxis.set_label("")

for ax in (ax5, ax6):
    ax.set_xticks([16, 8, 4, 2])
    ax.set_xlabel(r"$\Delta_i$ (m)")

out_path = Path(config.path.results)
out_path.mkdir(parents=True, exist_ok=True)
fig.savefig(out_path / "model_comparison_lcz_res_sensitivity.pdf", dpi=300)

# XY-cross sections of $w$ and $\theta$


Plot $xy$-cross sections of vertical velocity $w$ and potential temperature $\theta$ at a snapshot time as an example for the manuscript.


In [ ]:
snapshot_time = np.datetime64("2018-03-31T12:00")
reference_height = 42.0

In [ ]:
usm_fine_thetav = (
    (
        outputs[usm_run]["fine"]["xy"]["theta_xy"]
        * (0.608 * outputs[usm_run]["fine"]["xy"]["q_xy"] + 1)
    )
    .sel(time=snapshot_time, method=r"nearest")
    .sel(zu_xy=reference_height, method="nearest")
    .where(total_target_area_filter, drop=True)
    .squeeze()
)

usm_coarse_thetav = (
    (
        outputs[usm_run]["coarse"]["xy"]["theta_xy"]
        * (0.608 * outputs[usm_run]["coarse"]["xy"]["q_xy"] + 1)
    )
    .sel(time=snapshot_time, method=r"nearest")
    .sel(zu_xy=reference_height, method="nearest")
    .where(total_target_area_filter, drop=True)
    .squeeze()
)

slurb_coarse_thetav = (
    (
        outputs[slurb_run]["coarse"]["xy"]["theta_xy"]
        * (0.608 * outputs[slurb_run]["coarse"]["xy"]["q_xy"] + 1)
    )
    .sel(time=snapshot_time, method=r"nearest")
    .sel(zu_xy=reference_height, method="nearest")
    .where(total_target_area_filter, drop=True)
    .squeeze()
)

vmin = min(
    map(
        float,
        [usm_fine_thetav.min(), usm_coarse_thetav.min(), slurb_coarse_thetav.min()],
    )
)
vmax = max(
    map(
        float,
        [usm_fine_thetav.max(), usm_coarse_thetav.max(), slurb_coarse_thetav.max()],
    )
)

In [ ]:
fig = plt.figure(figsize=(FIGURE_WIDTH_L, 10.5 / 2.56), layout="constrained")
gs = fig.add_gridspec(
    2, 4, height_ratios=[1, 1], width_ratios=[1, 1, 1, 0.08], wspace=0.0, hspace=0.0
)

cbar1_ax = fig.add_subplot(gs[0, 3])
# cbar1_ax.set_axis_off()

cbar2_ax = fig.add_subplot(gs[1, 3])
# cbar2_ax.set_axis_off()


plt.colorbar(
    mpl.cm.ScalarMappable(
        norm=mpl.colors.Normalize(vmin=-2.5, vmax=2.5), cmap="RdBu_r"
    ),
    cax=cbar1_ax,
    orientation="vertical",
    label=r"$w~(\mathrm{m}~\mathrm{s}^{-1})$",
    extend="both",
    ticks=[-2, -1, 0, 1, 2],
)

plt.colorbar(
    mpl.cm.ScalarMappable(
        norm=mpl.colors.Normalize(vmin=vmin, vmax=vmax), cmap="viridis"
    ),
    cax=cbar2_ax,
    orientation="vertical",
    label=r"$\theta_v~(\mathrm{K})$",
    ticks=np.arange(286, 289, 0.5),
)

ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[0, 2])
ax4 = fig.add_subplot(gs[1, 0])
ax5 = fig.add_subplot(gs[1, 1])
ax6 = fig.add_subplot(gs[1, 2])

ax1.set_title(r"SLUrb, $\Delta_i=16~\mathrm{m}$")
ax2.set_title(r"Resolved, $\Delta_i=16~\mathrm{m}$")
ax3.set_title(r"Resolved, $\Delta_i=2~\mathrm{m}$")


outputs[usm_run]["fine"]["xy"]["w_xy"].sel(time=snapshot_time, method=r"nearest").sel(
    zw_xy=reference_height, method="nearest"
).where(total_target_area_filter, drop=True).squeeze().plot.imshow(
    ax=ax3, cmap="RdBu_r", add_colorbar=False, add_labels=False, vmin=-2.5, vmax=2.5
)
outputs[usm_run]["coarse"]["xy"]["w_xy"].sel(time=snapshot_time, method=r"nearest").sel(
    zw_xy=reference_height, method="nearest"
).where(total_target_area_filter, drop=True).squeeze().plot.imshow(
    ax=ax2, cmap="RdBu_r", add_colorbar=False, add_labels=False, vmin=-2.5, vmax=2.5
)
outputs[slurb_run]["coarse"]["xy"]["w_xy"].sel(
    time=snapshot_time, method=r"nearest"
).sel(zw_xy=reference_height, method="nearest").where(
    total_target_area_filter, drop=True
).squeeze().plot.imshow(
    ax=ax1, cmap="RdBu_r", add_colorbar=False, add_labels=False, vmin=-2.5, vmax=2.5
)


usm_fine_thetav.plot.imshow(
    ax=ax6, add_colorbar=False, add_labels=False, vmin=vmin, vmax=vmax
)
usm_coarse_thetav.plot.imshow(
    ax=ax5, add_colorbar=False, add_labels=False, vmin=vmin, vmax=vmax
)
slurb_coarse_thetav.plot.imshow(
    ax=ax4, add_colorbar=False, add_labels=False, vmin=vmin, vmax=vmax
)


ax1.add_artist(moffsetbox.AnchoredText("(a)", loc="upper left", pad=0.2, borderpad=0))
ax2.add_artist(moffsetbox.AnchoredText("(b)", loc="upper left", pad=0.2, borderpad=0))
ax3.add_artist(moffsetbox.AnchoredText("(c)", loc="upper left", pad=0.2, borderpad=0))
ax4.add_artist(moffsetbox.AnchoredText("(d)", loc="upper left", pad=0.2, borderpad=0))
ax5.add_artist(moffsetbox.AnchoredText("(e)", loc="upper left", pad=0.2, borderpad=0))
ax6.add_artist(moffsetbox.AnchoredText("(f)", loc="upper left", pad=0.2, borderpad=0))

xlims = (total_target_area_bbox[0], total_target_area_bbox[2])
ylims = (total_target_area_bbox[1], total_target_area_bbox[3])

for ax in (ax1, ax2, ax3, ax4, ax5, ax6):
    ax.set_aspect("equal")
    ax.set_xticks(np.arange(2200, 4001, 200))
    ax.set_yticks(np.arange(1400, 2601, 200))
    ax.set_xlabel("$x$ (m)")
    ax.set_ylabel("$y$ (m)")
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)

for ax in (ax2, ax3, ax5, ax6):
    ax.set_ylabel("")
    ax.set_yticklabels([])

for ax in (ax1, ax2, ax3):
    ax.set_xlabel("")
    ax.set_xticklabels([])

out_path = Path(config.path.results)
out_path.mkdir(parents=True, exist_ok=True)
fig.savefig(out_path / "model_comparison_xy_example.pdf", dpi=300)